<a href="https://colab.research.google.com/github/yookkk1/stats/blob/master/quickstarts/Prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U scikeras tensorflow scikit-learn==1.5.2


In [ ]:
# ============================================
# 0. Install Library (jalankan sekali di Colab)
# ============================================
!pip install scikeras tensorflow

# ============================================
# 1. Import Library
# ============================================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, f1_score, ConfusionMatrixDisplay
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from google.colab import files

# ============================================
# 2. Upload File CSV
# ============================================
print("Silakan upload file CSV Anda (contoh: Prakiraan_Cuaca.csv):")
uploaded = files.upload()
data = pd.read_csv(list(uploaded.keys())[0])

# ============================================
# 3. Persiapan Data
# ============================================
label_map = {"Hujan": 0, "Cerah": 1, "Berawan": 2}
data["cuaca"] = data["cuaca"].map(label_map)

X = data[["suhu", "kelembaban", "angin"]].dropna()
y = data.loc[X.index, "cuaca"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=42, stratify=y
)

# ============================================
# 4. Normalisasi
# ============================================
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# ============================================
# 5. Definisi Model
# ============================================
def create_model(neurons=32, dropout_rate=0.2):
    model = Sequential([
        Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)),
        Dropout(dropout_rate),
        Dense(neurons // 2, activation='relu'),
        Dense(3, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# ============================================
# 6. Training dengan GridSearch
# ============================================
model = KerasClassifier(
    model=create_model,
    verbose=0,
    callbacks=[EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)]
)

param_grid = {
    'model__neurons': [16, 32],
    'model__dropout_rate': [0.0, 0.2],
    'batch_size': [16],
    'epochs': [20]
}

grid = GridSearchCV(model, param_grid, scoring='f1_macro', cv=2, verbose=1)
grid.fit(X_train, y_train)

print("\nBest Score:", grid.best_score_)
print("Best Params:", grid.best_params_)

# ============================================
# 7. Evaluasi Model
# ============================================
y_pred = grid.best_estimator_.predict(X_test)
if y_pred.ndim > 1:
    y_pred = np.argmax(y_pred, axis=1)

print("\nAkurasi:", f"{accuracy_score(y_test, y_pred):.4f}")
print("F1 Macro:", f"{f1_score(y_test, y_pred, average='macro'):.4f}")
print("\nLaporan Klasifikasi:")
print(classification_report(y_test, y_pred, target_names=list(label_map.keys())))

ConfusionMatrixDisplay.from_predictions(y_test, y_pred, display_labels=list(label_map.keys()))
plt.title("Confusion Matrix - Neural Network")
plt.show()
